In [2]:
import os
import numpy as np
import pandas as pd
import math
import torch
import numpy as np
from nats_bench import create
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn import functional as F
import xautodl
from xautodl.datasets.DownsampledImageNet import ImageNet16
from xautodl.datasets.SearchDatasetWrap import SearchDataset
from xautodl.config_utils import load_config
import torch
import torchvision.models as M
from torch.optim.lr_scheduler import CosineAnnealingLR
from xautodl.models import get_cell_based_tiny_net

#from datatest import get_valid_test_loader, ECELoss, AdaptiveECELoss, ClasswiseECELoss, get_logits_labels
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn, optim
from torch.nn import functional as F
from torch.optim import Optimizer

In [3]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2070 with Max-Q Design'

In [4]:
def get_train_loader(dataset, batch):
    data_dir = 'D:\\Datasets\\Cifar\\cifar-10-python\\cifar-10-python'

    if dataset == "cifar10":
        normalize = transforms.Normalize(
            mean=[x / 255 for x in [125.3, 123.0, 113.9]],
            std=[x / 255 for x in [63.0, 62.1, 66.7]],
        )

        # define transform
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, padding=4),
            transforms.ToTensor(),
            normalize,
        ])
        data = datasets.CIFAR10(
            root=data_dir, train=True,
            download=True, transform=transform,
        )

        num_train = len(data)
        # print("num_train:{}".format(num_train))
        indices = list(range(num_train))
        split = int(np.floor(0.1 * num_train))
        # split = 0

        np.random.seed(1)
        np.random.shuffle(indices)

        train_idx, valid_idx = indices[split:], indices[:split]

        train_sampler = SubsetRandomSampler(train_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)

        train_loader = torch.utils.data.DataLoader(
            data, batch_size=batch, sampler=train_sampler,
            num_workers=1, pin_memory=True,
        )
        valid_loader = torch.utils.data.DataLoader(
            data, batch_size=batch, sampler=valid_sampler,
            num_workers=1, pin_memory=True,
        )

    return train_loader, valid_loader


def get_test_loader(batch_size,
                    shuffle=True,
                    num_workers=1,
                    pin_memory=True):
    normalize = transforms.Normalize(
        mean=[x / 255 for x in [125.3, 123.0, 113.9]],
        std=[x / 255 for x in [63.0, 62.1, 66.7]],
    )

    # define transform
    transform = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])

    data_dir = 'D:\\Datasets\\Cifar\\cifar-10-python\\cifar-10-python'
    # data_dir = '/media/linwei/disk1/NATS-Bench/cifar.python'
    dataset = datasets.CIFAR10(
        root=data_dir, train=False,
        download=True, transform=transform,
    )

    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory,
    )

    return data_loader

def get_logits_labels(data_loader, net):
    logits_list = []
    labels_list = []
    net.eval()
    for data, label in data_loader:
        data = data.cuda(0)
        logits = net(data)
        logits_list.append(logits)
        labels_list.append(label)
        logits = torch.cat(logits_list).cpu()
        labels = torch.cat(labels_list).cpu()
    return logits, labels

In [5]:
url = 'D:\\Datasets\\Cifar\\cifar-10-python\\cifar-10-python'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
dset = 'cifar10'

testloader = get_test_loader(batch_size=256)
trainloader, valloader = get_train_loader(dset, batch=256)

model = M.resnet50(pretrained=True)
dim_in = model.fc.in_features
model.fc = nn.Linear(in_features=dim_in, out_features=100, bias=True)
model = model.cuda(0)

epochs = 350
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=0.0005, momentum=0.9, nesterov=True)

warmups = 20 * len(trainloader)

lamda = lambda curiter: curiter / warmups if curiter < warmups else \
    (0.001 + 0.5 * 0.9 * (1.0 + math.cos((curiter - warmups) / ((epochs - 20) * len(trainloader)) * math.pi)))
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lamda)
lossfunction = nn.CrossEntropyLoss()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting D:\Datasets\Cifar\cifar-10-python\cifar-10-python\cifar-10-python.tar.gz to D:\Datasets\Cifar\cifar-10-python\cifar-10-python
Files already downloaded and verified


D:\Anaconda\envs\ml\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\envs\ml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\36101/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [6]:
for i, (input, label) in enumerate(trainloader):
    a = input
    b = label
    break

In [7]:
logits = model(a.cuda(0))

In [8]:
logits[0]

tensor([ 0.1604,  0.8685,  0.2373, -0.4165, -0.1285,  0.7005,  0.1664, -0.1828,
        -0.3752, -0.9446, -0.2734, -0.6405,  0.6580, -0.0781,  0.3323, -0.6156,
         0.0332, -0.5873, -0.3022,  0.1321,  0.0157,  0.5818, -0.4021, -0.0373,
        -0.0191, -0.5799, -0.3191, -0.6725,  0.3245,  0.3147, -0.2908, -0.3334,
         0.6438,  0.2544,  0.4905,  0.0592, -0.3508,  1.0222, -0.6164, -0.3681,
         0.4904, -0.0394, -0.3722,  0.4979,  0.5271, -0.0428,  0.5845, -0.0318,
         0.0426, -0.1796,  0.6308, -0.0752, -0.1816, -0.2019, -0.5120,  0.2559,
        -0.5318, -0.4357, -0.6026,  0.3086, -0.8183, -0.0047, -0.5794, -0.3625,
        -0.9270,  0.4722,  0.8223, -0.4421, -0.2141, -0.5101, -0.2315, -0.3339,
         0.3589,  0.0891, -0.0574, -0.5132,  0.0861, -0.4465,  0.2206,  0.5878,
        -0.3854, -0.3552,  0.4350,  0.0663,  0.5761,  0.1743, -0.2976, -0.5073,
        -0.4644, -0.1930, -0.2676, -0.5512,  0.1851, -0.7310,  0.3737, -0.0283,
        -0.2826, -0.1881, -0.2597, -0.42

In [9]:
loss = lossfunction(logits, b.cuda(0))

In [10]:
best_val_acc = 0
best_model_dict = {}
ep_num = 0
val_acc = []
test_acc = []
ep_list = []
for epoch in range(epochs):
    ep_list.append(epoch)
    #print(ep_list)
    for i, (input, label) in enumerate(trainloader):
        model.train()
        input = input.cuda(0)
        label = label.cuda(0)
        optimizer.zero_grad()

        logits = model(input)
        #print(logits)
        loss = lossfunction(logits, label)

        loss.backward()
        optimizer.step()
        scheduler.step()
        
    model.eval()
    with torch.no_grad():
        logits, labels = get_logits_labels(valloader, model)
        softmaxes = F.softmax(logits, dim=0)
        _, predictions = torch.max(softmaxes, 1)
        acc = predictions.eq(labels).float().mean().item()
        print("Current epoch: {}, Val_acc:{}".format(epoch+1, acc))

Current epoch: 1, Val_acc:0.00860000029206276
Current epoch: 2, Val_acc:0.13860000669956207
Current epoch: 3, Val_acc:0.03460000082850456
Current epoch: 4, Val_acc:0.008200000040233135
Current epoch: 5, Val_acc:0.004399999976158142
Current epoch: 6, Val_acc:0.009800000116229057


KeyboardInterrupt: 

In [13]:
with torch.no_grad():
    info = []
    logits, labels = get_logits_labels(testloader, model)
    softmaxes = F.softmax(logits, dim=0)
    _, predictions = torch.max(softmaxes, 1)
    nacc = predictions.eq(labels).float().mean()
    info.append(nacc)
    ece_criterion = ECELoss().cuda(0)
    nece = ece_criterion(logits, labels).item()
    info.append(nece)

    print(info)

[tensor(0.3524), 0.049374282360076904]


In [12]:
class ECELoss(nn.Module):
    '''
    Compute ECE (Expected Calibration Error)
    '''
    def __init__(self, n_bins=15):
        super(ECELoss, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels):
        softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)

        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

        return ece

In [16]:
class ClasswiseECELoss(nn.Module):
    '''
    Compute Classwise ECE
    '''
    def __init__(self, n_bins=15):
        super(ClasswiseECELoss, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels):
        num_classes = int((torch.max(labels) + 1).item())
        softmaxes = F.softmax(logits, dim=1)
        per_class_sce = None

        for i in range(num_classes):
            class_confidences = softmaxes[:, i]
            class_sce = torch.zeros(1, device=logits.device)
            labels_in_class = labels.eq(i) # one-hot vector of all positions where the label belongs to the class i

            for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
                in_bin = class_confidences.gt(bin_lower.item()) * class_confidences.le(bin_upper.item())
                prop_in_bin = in_bin.float().mean()
                if prop_in_bin.item() > 0:
                    accuracy_in_bin = labels_in_class[in_bin].float().mean()
                    avg_confidence_in_bin = class_confidences[in_bin].mean()
                    class_sce += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

            if (i == 0):
                per_class_sce = class_sce
            else:
                per_class_sce = torch.cat((per_class_sce, class_sce), dim=0)

        sce = torch.mean(per_class_sce)
        return sce

In [21]:
class AdaptiveECELoss(nn.Module):
    '''
    Compute Adaptive ECE
    '''
    def __init__(self, n_bins=15):
        super(AdaptiveECELoss, self).__init__()
        self.nbins = n_bins

    def histedges_equalN(self, x):
        npt = len(x)
        return np.interp(np.linspace(0, npt, self.nbins + 1),
                     np.arange(npt),
                     np.sort(x))
    def forward(self, logits, labels):
        softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)
        n, bin_boundaries = np.histogram(confidences.cpu().detach(), self.histedges_equalN(confidences.cpu().detach()))
        #print(n,confidences,bin_boundaries)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]
        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
        return ece

In [18]:
with torch.no_grad():
    info = []
    logits, labels = get_logits_labels(testloader, model)
    softmaxes = F.softmax(logits, dim=0)
    _, predictions = torch.max(softmaxes, 1)
    nacc = predictions.eq(labels).float().mean()
    info.append(nacc)
    ece_criterion = ClasswiseECELoss().cuda(0)
    nece = ece_criterion(logits, labels).item()
    info.append(nece)

    print(info)

[tensor(0.3524), 0.014100000262260437]


In [22]:
with torch.no_grad():
    info = []
    logits, labels = get_logits_labels(testloader, model)
    softmaxes = F.softmax(logits, dim=0)
    _, predictions = torch.max(softmaxes, 1)
    nacc = predictions.eq(labels).float().mean()
    info.append(nacc)
    ece_criterion = AdaptiveECELoss().cuda(0)
    nece = ece_criterion(logits, labels).item()
    info.append(nece)

    print(info)

[tensor(0.3524), 0.04938226193189621]


In [33]:
class MECELoss(nn.Module):
    '''
    Compute MECE (Expected Calibration Error)
    '''
    def __init__(self, n_bins=15):
        super(MECELoss, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels):
        softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)

        ece = torch.zeros(1, device=logits.device)
        mece = []
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
                mece.append(torch.abs(avg_confidence_in_bin - accuracy_in_bin))
        print(mece)

        return max(mece)

In [35]:
with torch.no_grad():
    info = []
    logits, labels = get_logits_labels(testloader, model)
    softmaxes = F.softmax(logits, dim=0)
    _, predictions = torch.max(softmaxes, 1)
    nacc = predictions.eq(labels).float().mean()
    info.append(nacc)
    ece_criterion = MECELoss().cuda(0)
    nece = ece_criterion(logits, labels).item()
    info.append(nece)

    print(info)

[tensor(0.0643), tensor(0.1126), tensor(0.0517), tensor(0.0376), tensor(0.0434), tensor(0.1208), tensor(0.0593), tensor(0.1137), tensor(0.0727), tensor(0.0910), tensor(0.0989), tensor(0.0244)]
[tensor(0.3523), 0.12081539630889893]
